In [1]:
from datasets import load_dataset

dataset = load_dataset("tiedong/goat")

Found cached dataset json (/home/diwank/.cache/huggingface/datasets/tiedong___json/tiedong--goat-55b7467c033a1462/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
dataset = dataset["train"]

In [3]:
def add_signature(row):
    output = row["output"]
    signature = "".join(sorted(set(output)))
    return dict(signature=signature)

dataset = dataset.map(add_signature)

Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/tiedong___json/tiedong--goat-55b7467c033a1462/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-2a0bcb4de0d0fc73.arrow


In [4]:
sigs = set(dataset["signature"])

In [5]:
deduplicated = {}

for row in dataset:
    if len(deduplicated) >= len(sigs):
        break

    deduplicated[row["signature"]] = row

len(deduplicated)

6652

In [6]:
from datasets import Dataset

dd_dataset = Dataset.from_list(list(deduplicated.values()))

In [7]:
dd_dataset.push_to_hub("diwank/goat-deduped", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.


In [8]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

In [9]:
def to_chatml(row):
    instruction = row["instruction"]
    output = row["output"]
    answer = row["answer"]

    chatml = [
        person(instruction),
        thought(f"To solve this, we must do:\n{output}"),
        me(f"The answer is {answer}"),
    ]

    return dict(chatml=chatml)

processed_dataset = dd_dataset.map(to_chatml)

Map:   0%|          | 0/6652 [00:00<?, ? examples/s]

In [10]:
processed_dataset.remove_columns(list(set(processed_dataset.column_names) - {"chatml"})).push_to_hub("diwank/goat-chatml", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
processed_dataset[0]["chatml"]

[{'content': '51286031623+9630381 =', 'name': None, 'role': 'user'},
 {'content': 'To solve this, we must do:\n51286031623 + 9630381 = 51295662004',
  'name': 'thought',
  'role': 'system'},
 {'content': 'The answer is 51295662004', 'name': None, 'role': 'assistant'}]